In [1]:
import pandas as pd
#PRE PROCESS DATA TO FORMAT BEFORE UPLOADING
# Load the data from the uploaded CSV file
file_path = './export.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head(), data.columns


(         Date                Description   Category     Cost Currency  \
 0  2024-06-11              Train Tickets  Bus/train  1559.40      CAD   
 1  2024-06-11  Abhishek M. paid Abdullah    Payment   389.85      CAD   
 2  2024-06-11    Darpan K. paid Abdullah    Payment   389.85      CAD   
 3  2024-07-02     Rabea A. paid Abdullah    Payment   389.85      CAD   
 4  2024-07-12                       Uber       Taxi    14.18      CAD   
 
    Gurpartap  Darpan Khanna  Abdullah  Rabea Ahmed  Abhishek M (removed)  
 0        0.0        -389.85   1169.55      -389.85               -389.85  
 1        0.0           0.00   -389.85         0.00                389.85  
 2        0.0         389.85   -389.85         0.00                  0.00  
 3        0.0           0.00   -389.85       389.85                  0.00  
 4        0.0          -4.73      9.46        -4.73                  0.00  ,
 Index(['Date', 'Description', 'Category', 'Cost', 'Currency', 'Gurpartap',
        'Darpan Khann

In [16]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = './export.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(data.head())


             Description  Gurpartap  Darpan Khanna  Abdullah  Rabea Ahmed
0                   Uber       0.00          -4.73      9.46        -4.73
1              Groceries       0.00         -42.18     84.36       -42.18
2                   Coke       6.64          -2.21     -2.21        -2.22
3  Darpan painting lease       0.00          20.00    -20.00         0.00
4                  Metro       0.00          -7.00      7.00         0.00


In [19]:
people = data.columns[1:]
balances = pd.DataFrame(0, index=people, columns=people)
people
balances

,Gurpartap,Darpan Khanna,Abdullah,Rabea Ahmed
Gurpartap,0,0,0,0
Darpan Khanna,0,0,0,0
Abdullah,0,0,0,0
Rabea Ahmed,0,0,0,0


In [26]:
# Correcting the balances calculation logic
balances = pd.DataFrame(0, index=people, columns=people)

# Exclude non-numeric data for safe operations
numeric_data = data.select_dtypes(include=[np.number])

for index, row in numeric_data.iterrows():
    payers = row[row > 0]  # Individuals who paid in this transaction
    payees = row[row < 0]  # Individuals who owe in this transaction
    total_paid = payers.sum()  # Total paid by payers in this transaction

    if not payees.empty and total_paid > 0:
        for payer in payers.index:
            for payee in payees.index:
                # Calculate the amount each payee owes to the payer based on their share of the total owed
                owe_amount = -payees[payee] * (payers[payer] / total_paid)
                balances.loc[payee, payer] += owe_amount

# Net off the balances between each pair
for person1 in people:
    for person2 in people:
        if person1 != person2:
            net_amount = balances.loc[person1, person2] - balances.loc[person2, person1]
            if net_amount > 0:
                balances.loc[person1, person2] = net_amount
                balances.loc[person2, person1] = 0
            else:
                balances.loc[person1, person2] = 0
                balances.loc[person2, person1] = -net_amount

# Convert balances to a descriptive matrix
description_matrix = balances.copy()
for person1 in people:
    for person2 in people:
        amount = description_matrix.loc[person1, person2]
        if amount > 0:
            description_matrix.loc[person1, person2] = f"{person1} owes {person2} ${amount:.2f}"
        else:
            description_matrix.loc[person1, person2] = ""

# Calculate total owed by each person (sum across rows)
description_matrix['Total Owed'] = description_matrix[people].applymap(lambda x: float(x.split('$')[-1]) if x else 0).sum(axis=1)

# Calculate total owed to each person (sum across columns)
description_matrix.loc['Total Owed To'] = description_matrix[people].applymap(lambda x: float(x.split('$')[-1]) if x else 0).sum()

description_matrix


,Gurpartap,Darpan Khanna,Abdullah,Rabea Ahmed,Total Owed
Gurpartap,,,,,0.00
Darpan Khanna,Darpan Khanna owes Gurpartap $61.62,,Darpan Khanna owes Abdullah $95.73,,157.35
Abdullah,Abdullah owes Gurpartap $48.66,,,,48.66
Rabea Ahmed,Rabea Ahmed owes Gurpartap $127.34,Rabea Ahmed owes Darpan Khanna $3.95,Rabea Ahmed owes Abdullah $61.05,,192.34
Total Owed To,237.62,3.95,156.78,0.0,NaN


In [11]:
net_balances

Gurpartap               475.24
Darpan Khanna          -306.80
Abdullah                216.24
Rabea Ahmed            -384.68
Abhishek M (removed)      0.00
dtype: float64

In [27]:
# Calculate the total expense of the trip
total_expense = numeric_data[people].apply(lambda x: x[x > 0]).sum().sum()
total_expense


778.81